In [345]:
# Matrice delle distanze usando una lista di liste
distanze = [
    [0, 7, 3, 6, 2, 9, 4, 8],  # Distanze dalla città 0
    [7, 0, 5, 1, 8, 4, 6, 3],  # Distanze dalla città 1
    [3, 5, 0, 2, 7, 6, 9, 4],  # Distanze dalla città 2
    [6, 1, 2, 0, 4, 3, 8, 7],  # Distanze dalla città 3
    [2, 8, 7, 4, 0, 5, 3, 6],  # Distanze dalla città 4
    [9, 4, 6, 3, 5, 0, 7, 2],  # Distanze dalla città 5
    [4, 6, 9, 8, 3, 7, 0, 1],  # Distanze dalla città 6
    [8, 3, 4, 7, 6, 2, 1, 0]   # Distanze dalla città 7
]


2 3 1 5 7 6 4 0 
2+1+4+2+1+3+2
##implementare codice per generare randomicamente le distanze fra le cittò

def getPossibleSwaps(lista):
    possibleStates = []
    for i in range(0, len(lista) , 1 ):
        for j in range(i, len(lista) , 1 ):
            if ( i !=j ):
                possibleStates.append((lista[i],lista[j]))
    return possibleStates




def switchCities(cities , positions):
    position1 = positions[0]
    position2 = positions[1]
    newIndexOfPosition2 = cities.index(position1)
    newIndexOfPosition1 = cities.index(position2)
    cities[newIndexOfPosition1]= position1
    cities[newIndexOfPosition2] = position2
    return cities

def stateScore(cities):
    score = 0
    for i in range(0 , len(cities)-1 , 1):
        score = score + distanze[cities[i]][cities[i+1]]
        if ( i+1 == len(cities)-1):##aggiungo la distanza per tornare a casa :)
            score = score + distanze[cities[i+1]][0]
    return score
    


    # Accesso alla distanza tra città 0 e città 1
print("Distanza tra città 0 e città 1:", distanze[0][1])  # Output: 7







Distanza tra città 0 e città 1: 7


In [347]:
stateScore([0,4,6,7,5,1,3,2])

18

In [349]:
print("Distanza tra città 0 e città 1:", distanze[0][5])  # Output: 7



Distanza tra città 0 e città 1: 9


In [351]:
stateScore([0,4,6,7,5,1,3,2])

18

In [353]:
class NeighborhoodNode:

    swap = None
    state = None
    eval = None
    next = None

    def __init__(self , state , swap , score ):
        self.state = state
        self.swap = swap
        self.eval = score
        self.next = None
        

In [355]:
class State:

    cities = None
    eval = None

    def __init__(self, cities):
        self.cities = cities
        self.eval  = stateScore(cities)
    

    

In [357]:
class Neighborhood:
     def __init__(self):
        self.__head = None
        self.__tail = None


     def add(self,node):
        if (self.__head == None): ## zero elementi
            self.__head = node
        elif ( self.__tail== None): ## un elemento
            
            if ( node.eval > self.__head.eval):
            
                self.__tail = node
            else:
                self.__tail= self.__head
                self.__head = node

            self.__head.next = self.__tail
        elif( node.eval <= self.__head.eval):##inserimento in testa 
            temp = self.__head
            self.__head= node
            node.next = temp
        elif ( node.eval >= self.__tail.eval ): ## inserimento in coda
            temp = self.__tail
            self.__tail = node
            temp.next = self.__tail
        else:  
            ## inserimento nel mezzo
            temp =  self.__head
            while ( temp.next is not None ):
                if ( node.eval > temp.eval and node.eval < temp.next.eval):
                        prec = temp
                        succ = temp.next
                        prec.next = node
                        node.next = succ
                        break
                else:
                    temp = temp.next

     def pop(self):

        temp = self.__head
        if ( temp.next == self.__tail ):
             self.__head = self.__tail
             self.__tail = None
        else:
            self.__head = temp.next
        
        return temp

     def print(self):
        node = self.__head
        while (node is not None ): 
            print(node.eval)
            node = node.next

     def isEmpty(self):
        if( self.__head == None ):
            return 1
        else:
            return 0

     def addList(self , nodes ):
        for node in nodes:
            self.add(node)

     def clean (self ) :
        self.head = None
        self.tail = None

            
    
    

In [359]:

class Problem:


    initialState = []
    finalState = []

    def __init__(self , initialState , finalState):
        self.initialState = initialState
        self.finalState = finalState


    def getNeighborhood(self, currentState, tabuList):
        neighborhood = Neighborhood()
         #ho uno stato , ovvero una lista di città
        #ho M che contiene i possibili swap collegati con i rispettivi tabu
        #ottengo i possibili swap senza tabu -> devo avere una struttura di dati che mi permette di filtrare attraverso il tabu tenure
        acceptableSwaps = tabuList.getAcceptableSwaps()    
        #applico gli swap ottenendo una serie di possibili stati
        for acceptableSwap in acceptableSwaps:
            newState = switchCities( currentState , acceptableSwap)
            neighborhood.add(NeighborhoodNode(newState, acceptableSwap,stateScore(newState)))
        
        return neighborhood


    def getNextState( self, N , M ):
        neighborhoodNode = N.pop()
        nextState = neighborhoodNode.state
        M.setTableTenureForANode(neighborhoodNode.swap)
        print(f"Il prossimo stato sarà:{nextState}, dovuto allo swap:{neighborhoodNode.swap} , con punteggio:{neighborhoodNode.eval}")
        return nextState

    def getInitialTabuList(self, tabuTenure):
        tl = TabuList(self.initialState , tabuTenure )
        tl.addList()
        return tl
        
    def getInitialNeighborhood(self):
        return Neighborhood()
        
    def getInitialState(self):
        return self.initialState

    def getFinalState(self):
        return self.finalState

                


In [361]:
class TabuList():

    citiesList = None
    head = None
    tableTenureValue = None
    
    def __init__(self, citiesList , tableTenureValue):
        self.head = None
        self.citiesList = citiesList
        self.tableTenureValue = tableTenureValue

    def add(self, elem):

        if ( self.head == None ):
            self.head = elem
        elif(self.head.next == None):
            self.head.next = elem
        else:
            node = self.head
            while ( node.next != None):
                node = node.next
                #arrivo a fine lista
            node.next = elem 

    def addList(self):
        #prendo la lista di città , genero tutti i possibilit stati , aggiungo un nodo alla tabu list creando progressivamente i nodi
        possibleSwaps = getPossibleSwaps(self.citiesList)
        for swap in possibleSwaps:
            self.add(TabuListElem(swap))


    def print(self):
        node = self.head
        while(node != None):
            print(f"Swap: {node.possibleSwap}, table tenure: {node.tableTenure}")
            node = node.next


    def updateTenure(self):
        nodo = self.head
        while( nodo != None):
            if ( nodo.tableTenure > 0 ):
                nodo.tableTenure = nodo.tableTenure - 1
            nodo = nodo.next

    def setTableTenureForANode(self, swap ):
        nodo = self.head
        while( nodo.possibleSwap != swap):
            nodo = nodo.next
        nodo.setTableTenure(self.tableTenureValue)

    def getAcceptableSwaps(self):
        acceptableStates = []
        nodo = self.head
        if ( nodo == None):
            print("tabu list vuota")
            return None
        while(nodo != None):
            if ( nodo.tableTenure == 0 ):
                acceptableStates.append(nodo.possibleSwap)
            nodo = nodo.next
        return acceptableStates
            

        

    

In [363]:
class TabuListElem:

    possibleSwap = None
    tableTenure = None
    next = None


    def __init__(self, possibleSwap):
        self.possibleSwap = possibleSwap
        self.tableTenure = 0
        self.next = None
        


    def setTableTenure(self,tableTenureValue):
        self.tableTenure = tableTenureValue



In [426]:
import random



def tabuSearch( problem ):
    tabuTenure = 15
    M = problem.getInitialTabuList(tabuTenure)
    N = problem.getInitialNeighborhood()
    currentState = problem.getInitialState()
    print(f"Stato iniziale:{currentState}")
    bestState = State( currentState )
    i = 1
    while ( currentState != problem.getFinalState()  ):
        print(f"{i}° Iterazione")
        i += 1
        M.updateTenure()
        ## N sarà implementato come una lista ordinata, che restituisce per primo l'elemento che ha una valutazione migliore , tenendo conto di M
        N = problem.getNeighborhood(currentState, M)
        currentState = problem.getNextState( N , M) # M viene aggiornato dentro il metodo prima di restituire la prossima mossa,  o meglio il prossimo stato 

        
        if ( stateScore(currentState) < bestState.eval ):
            bestState.cities = tuple(currentState)
            bestState.eval = stateScore(currentState)
            for i in range(1,10,1):
                print("")
            print("------------------Aggiornamento--------------------")
            print(f"Il best state è:{bestState.cities} con valutazione: {bestState.eval}")
            for i in range(1,10,1):
                print("")
            
        
            
            
       

    print(f"Trovata soluzione ottima: {bestState.cities}")
    
        

        

        

In [428]:
problem = Problem( [0,1,2,3,4,5,6,7] , [2,3,1,5,7,6,4,0])

tabuSearch(problem)

Stato iniziale:[0, 1, 2, 3, 4, 5, 6, 7]
1° Iterazione
Il prossimo stato sarà:[7, 6, 5, 4, 3, 2, 1, 0], dovuto allo swap:(5, 7) , con punteggio:26









------------------Aggiornamento--------------------
Il best state è:(7, 6, 5, 4, 3, 2, 1, 0) con valutazione: 31









9° Iterazione
Il prossimo stato sarà:[0, 1, 2, 3, 4, 6, 5, 7], dovuto allo swap:(1, 7) , con punteggio:32
10° Iterazione
Il prossimo stato sarà:[7, 5, 6, 4, 3, 2, 1, 0], dovuto allo swap:(1, 2) , con punteggio:22









------------------Aggiornamento--------------------
Il best state è:(7, 5, 6, 4, 3, 2, 1, 0) con valutazione: 30









9° Iterazione
Il prossimo stato sarà:[0, 1, 2, 3, 4, 6, 7, 5], dovuto allo swap:(0, 7) , con punteggio:30
10° Iterazione
Il prossimo stato sarà:[5, 7, 6, 4, 3, 0, 2, 1], dovuto allo swap:(0, 4) , con punteggio:29
11° Iterazione
Il prossimo stato sarà:[1, 2, 0, 4, 3, 5, 6, 7], dovuto allo swap:(0, 1) , con punteggio:19
12° Iterazione
Il prossimo stato sarà:[5, 6, 7, 4, 3, 1, 0,